In [30]:
#!/usr/bin/env python
# coding: utf-8


import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

DIR = 'MalariaDrugImagesGHS'
DIM = 128

count = len(os.listdir(DIR))
images = np.zeros([count, DIM, DIM], dtype='uint8')

# Read and resize images
scaler = StandardScaler()
for (i,image) in zip(range(len(os.listdir(DIR))), os.listdir(DIR)):
    im_arr = cv2.imread(os.path.join(DIR, image))
    im_arr = cv2.cvtColor(im_arr, cv2.COLOR_BGR2GRAY)
    im_arr = cv2.resize(im_arr, (DIM, DIM))
    im_arr = scaler.fit_transform(im_arr)
    plt.imshow(im_arr)
    images[i] = im_arr

images = images.reshape(count, -1)

'''
pca = PCA(n_components=200)
pca.fit(images)

'''
# pca = PCA(.99)
# lower_dimensional_data = pca.fit_transform(images)
# pca.n_components_


# In[89]:

lower_dimensional_data = None
for var in [0.9]: #, 0.9, 0.85, 0.8, 0.75, 0.3]:
    pca = PCA(var)
    pca.fit(images)
    lower_dimensional_data = pca.fit_transform(images)
    approximation = pca.inverse_transform(lower_dimensional_data)
    plt.figure(figsize=(8,4));

    # Original Image
    plt.subplot(1, 2, 1);
    plt.imshow(images[1].reshape(DIM,DIM),
                  cmap = plt.cm.gray, interpolation='nearest',
                  clim=(0, 255));
    plt.xlabel(str(images[0].shape[0]) + ' components', fontsize = 14)
    plt.title('Original Image', fontsize = 20);

    # 154 principal components
    plt.subplot(1, 2, 2);
    plt.imshow(approximation[1].reshape(DIM, DIM),
                  cmap = plt.cm.gray, interpolation='nearest',
                  clim=(0, 255));
    plt.xlabel(str(pca.n_components_) + ' components', fontsize = 14)
    plt.title(str(var*100) + '% of Explained Variance', fontsize = 20);
    plt.show()
    print('Keeping ' +str(var*100) +' percent of the original data requires: ' + str(pca.n_components_) + ' bytes')



Keeping 90.0 percent of the original data requires: 25 bytes


In [29]:
lower_dimensional_data[0].shape

(12,)

In [46]:
from qiskit import QuantumCircuit, execute, Aer, IBMQ
IBMQ.

In [59]:

# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)

# import settings

import os


class QBitsLoader():
    
    def __init__(self, online=False):
        self.online = online

        if self.online:
            TOKEN = os.getenv('TOKEN')
            IBMQ.save_account(TOKEN, overwrite=True)
            provider = IBMQ.load_account()
            self.backends = provider.backends()
            self.backends.append(Aer.backends())
        else:
            print('[INFO] Offline mode: only simulators can be accessed')
            self.backends = Aer.backends()

    def get_backends(self):
        """
            Returns a list of tuples with the available backends and their number of qubits
        """
        return [backend.name() for backend in self.backends]

    def get_backends_with_qubits(self):
        """
            Returns a list of tuples with the available backends and their number of qubits
        """
        return [(backend.name(), backend.configuration().n_qubits,) for backend in self.backends]
    
    def find_backend(self, name):
        """
            Finds a backend by its name
        """
        for backend in self.backends:
            if name == backend.name():
                return backend
        raise ValueError(f"Backend '{name}' not found")
    
    def get_backend_qubits(self, name):
        """
            Returns the number of qubits a certain quantum device can handle
        """
        return self.find_backend(name).configuration.n_qubits
    
    def pca_recommendation(self, name):
        qubits = self.get_backend_qubits(name)
        components = 2**qubits
    
    

if __name__ == "__main__":
    q_bits_loader = QBitsLoader(False)
    
    print(q_bits_loader.get_backends())
    print(q_bits_loader.get_backends_with_qubits())
    b = q_bits_loader.call_backend('unitary_simulator')

[INFO] Offline mode: only simulators can be accessed
['qasm_simulator', 'statevector_simulator', 'unitary_simulator']
[('qasm_simulator', 29), ('statevector_simulator', 29), ('unitary_simulator', 14)]


In [55]:
import numpy as np
import math
import matplotlib.pyplot as plt 
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.aqua.components.initial_states import Custom
from qiskit import ClassicalRegister, Aer, execute, QuantumCircuit, QuantumRegister

def circuit(vec):
    qbits =  int(math.log2(length))
    cct = Custom(qbits, state_vector=vec)
    return cct.construct_circuit('circuit'), qbits

for image,feature in zip(os.listdir(DIR)[:3],lower_dimensional_data[:3]):
    vector = np.array(feature)
    length = vector.shape[0]
    if int(math.log2(length)) != length:
        pad_length = 2**(int(math.log2(length))+1)
        vector = np.pad(vector, (0, pad_length - length), mode='constant')
    length = vector.shape[0]
    
    cr, qbits = circuit(vector)
#     print(list(range(qbits)))
#     cr.add_register(ClassicalRegister(qbits))
#     cr.measure(list(range(qbits)), list(range(qbits)))
    
#     simulator = Aer.get_backend('qasm_simulator')
#     counts = execute(cr, backend=simulator, shots=1024).result().get_counts()
#     plot_histogram(counts)
    simulator = Aer.get_backend('statevector_simulator')
    result = execute(cr, backend = simulator).result()
    statevector = result.get_statevector()
    plot_bloch_multivector(statevector)
    cr.draw(output = 'mpl')

    plt.show()

    # 
    # print(dir(cr))

In [3]:
counts.values()

dict_values([1, 59, 404, 68, 35, 5, 64, 105, 95, 2, 186])

In [4]:
%matplotlib

Using matplotlib backend: Qt5Agg
